# Solarcast project

As part of the OMDENA MLOps course, we haD to work on a solar radiation project.
Solar and weather data from 2019 to 2021 for Boulogne - Billancourt (situated on the outskirts of Paris) was extracted from the Solarcast website https://toolkit.solcast.com.au/live-forecast

In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## I - Import dataset and a brief look

First we will import our dataset set

In [2]:
# Run cell if using Google colab
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
df = pd.read_csv("48.843519_2246138_Solcast_PT60M.csv")

In [4]:
# Visualize the first rows of our dataframe
df.head(5)

,PeriodEnd,PeriodStart,Period,AirTemp,CloudOpacity,Dhi,Dni,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m
0,2018-12-31T02:00:00Z,2018-12-31T01:00:00Z,PT60M,7.8,0.0,0,0,0,17.6,98.1,1021.2,282,2.6
1,2018-12-31T03:00:00Z,2018-12-31T02:00:00Z,PT60M,7.7,0.0,0,0,0,17.0,98.4,1021.1,284,2.7
2,2018-12-31T04:00:00Z,2018-12-31T03:00:00Z,PT60M,7.6,0.0,0,0,0,16.6,98.5,1020.9,288,2.6
3,2018-12-31T05:00:00Z,2018-12-31T04:00:00Z,PT60M,7.6,0.0,0,0,0,16.5,98.5,1020.7,296,2.5
4,2018-12-31T06:00:00Z,2018-12-31T05:00:00Z,PT60M,7.5,6.8,0,0,0,16.4,98.5,1020.5,303,2.5


In [5]:
# ... and bottom rows
df.tail(5)

,PeriodEnd,PeriodStart,Period,AirTemp,CloudOpacity,Dhi,Dni,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m
26080,2021-12-21T18:00:00Z,2021-12-21T17:00:00Z,PT60M,2.2,26.5,0,0,0,4.0,59.4,1011.9,75,2.5
26081,2021-12-21T19:00:00Z,2021-12-21T18:00:00Z,PT60M,1.7,26.1,0,0,0,4.0,62.2,1012.2,79,2.4
26082,2021-12-21T20:00:00Z,2021-12-21T19:00:00Z,PT60M,1.4,30.0,0,0,0,4.0,63.8,1012.5,80,2.3
26083,2021-12-21T21:00:00Z,2021-12-21T20:00:00Z,PT60M,1.1,29.9,0,0,0,4.0,64.7,1012.6,83,2.4
26084,2021-12-21T22:00:00Z,2021-12-21T21:00:00Z,PT60M,0.8,28.8,0,0,0,3.9,65.4,1012.4,88,2.6


A quick look at the characteristics of our dataset

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26085 entries, 0 to 26084
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PeriodEnd          26085 non-null  object 
 1   PeriodStart        26085 non-null  object 
 2   Period             26085 non-null  object 
 3   AirTemp            26085 non-null  float64
 4   CloudOpacity       26085 non-null  float64
 5   Dhi                26085 non-null  int64  
 6   Dni                26085 non-null  int64  
 7   Ghi                26085 non-null  int64  
 8   PrecipitableWater  26085 non-null  float64
 9   RelativeHumidity   26085 non-null  float64
 10  SurfacePressure    26085 non-null  float64
 11  WindDirection10m   26085 non-null  int64  
 12  WindSpeed10m       26085 non-null  float64
dtypes: float64(6), int64(4), object(3)
memory usage: 2.6+ MB


Most variables in our table are numeric variables to the exception  of date variables that are formatted as objects. However the date variables should be transformed into date time format.
Some of integer should actually be float as they taken on decimal values.

There seems to be no missing values in our dataset but we will confirm it later

In [7]:
print(f"Our dataset contains {df.shape[0]} observations and {df.shape[1]} columns")

Our dataset contains 26085 observations and 13 columns


In [8]:
# Print the dataset describe statistics (only numerical variables)
df.describe()

,AirTemp,CloudOpacity,Dhi,Dni,Ghi,PrecipitableWater,RelativeHumidity,SurfacePressure,WindDirection10m,WindSpeed10m
count,26085.000000,26085.000000,26085.000000,26085.000000,26085.000000,26085.000000,26085.000000,26085.000000,26085.000000,26085.000000
mean,12.971022,33.689469,73.893540,132.217175,143.573816,17.541848,67.561338,1005.030021,185.433698,3.617439
std,7.023319,30.067863,110.310656,251.149095,217.184033,7.734493,16.917940,9.563382,101.407703,1.909283
min,-5.400000,0.000000,0.000000,0.000000,0.000000,1.900000,12.100000,960.000000,0.000000,0.000000
25%,7.600000,0.700000,0.000000,0.000000,0.000000,11.400000,55.500000,999.900000,88.000000,2.200000
50%,12.400000,31.300000,6.000000,0.000000,7.000000,16.500000,69.700000,1006.000000,209.000000,3.300000
75%,17.900000,57.700000,112.000000,113.000000,228.000000,22.600000,80.800000,1011.300000,260.000000,4.700000
max,41.400000,99.500000,548.000000,986.000000,930.000000,47.400000,99.200000,1035.200000,360.000000,13.900000


In [9]:
#Print descriptive statistics for categorical variables
df.describe(include="object")

,PeriodEnd,PeriodStart,Period
count,26085,26085,26085
unique,26085,26085,1
top,2018-12-31T02:00:00Z,2018-12-31T01:00:00Z,PT60M
freq,1,1,26085


In [10]:
df.columns

Index(['PeriodEnd', 'PeriodStart', 'Period', 'AirTemp', 'CloudOpacity', 'Dhi',
       'Dni', 'Ghi', 'PrecipitableWater', 'RelativeHumidity',
       'SurfacePressure', 'WindDirection10m', 'WindSpeed10m'],
      dtype='object')

We will filter out the non relevant columns

In [11]:
# select some features
df= df[['PeriodEnd', 'PeriodStart', 'AirTemp', 'Dhi', 'Dni', 'Ghi',
       'PrecipitableWater', 'RelativeHumidity', 'SurfacePressure',
       'WindDirection10m', 'WindSpeed10m']]

In [12]:
# Rename column names for more clarity
old_names= ['PeriodEnd', 'PeriodStart', 'AirTemp', 'Dhi', 'Dni', 'Ghi','PrecipitableWater', 'RelativeHumidity',
             'SurfacePressure', 'WindDirection10m', 'WindSpeed10m'] 
new_names= ['PeriodEnd','PeriodStart','Temperature', 'Radiation', 'DNI', 'GHI', 'Precipitation', 'Humidity', 
             'Pressure', 'WindDirection', 'WindSpeed']
df.rename(columns= dict(zip(old_names, new_names)), inplace=True)

A quick sanity check to make sure the column name have been changed accordingly

In [13]:
df.head(5)

,PeriodEnd,PeriodStart,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,WindSpeed
0,2018-12-31T02:00:00Z,2018-12-31T01:00:00Z,7.8,0,0,0,17.6,98.1,1021.2,282,2.6
1,2018-12-31T03:00:00Z,2018-12-31T02:00:00Z,7.7,0,0,0,17.0,98.4,1021.1,284,2.7
2,2018-12-31T04:00:00Z,2018-12-31T03:00:00Z,7.6,0,0,0,16.6,98.5,1020.9,288,2.6
3,2018-12-31T05:00:00Z,2018-12-31T04:00:00Z,7.6,0,0,0,16.5,98.5,1020.7,296,2.5
4,2018-12-31T06:00:00Z,2018-12-31T05:00:00Z,7.5,0,0,0,16.4,98.5,1020.5,303,2.5


In [14]:
 # Interpret columns as appropriate data types to ensure compatibility
df['Radiation']     = df['Radiation'].astype(float)
df['Temperature']   = df['Temperature'].astype(float) # or int
df['Pressure']      = df['Pressure'].astype(float)
df['Humidity']      = df['Humidity'].astype(int) # or int
df['WindDirection'] = df['WindDirection'].astype(float)
df['WindSpeed']     = df['WindSpeed'].astype(float)
df['PeriodStart'] = pd.to_datetime(df['PeriodStart']).dt.to_period('T').dt.to_timestamp()
df['PeriodEnd'] = pd.to_datetime(df['PeriodEnd']).dt.to_period('T').dt.to_timestamp()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26085 entries, 0 to 26084
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   PeriodEnd      26085 non-null  datetime64[ns]
 1   PeriodStart    26085 non-null  datetime64[ns]
 2   Temperature    26085 non-null  float64       
 3   Radiation      26085 non-null  float64       
 4   DNI            26085 non-null  int64         
 5   GHI            26085 non-null  int64         
 6   Precipitation  26085 non-null  float64       
 7   Humidity       26085 non-null  int64         
 8   Pressure       26085 non-null  float64       
 9   WindDirection  26085 non-null  float64       
 10  WindSpeed      26085 non-null  float64       
dtypes: datetime64[ns](2), float64(6), int64(3)
memory usage: 2.2 MB


In [16]:
df.head(5)

,PeriodEnd,PeriodStart,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,WindSpeed
0,2018-12-31 02:00:00,2018-12-31 01:00:00,7.8,0.0,0,0,17.6,98,1021.2,282.0,2.6
1,2018-12-31 03:00:00,2018-12-31 02:00:00,7.7,0.0,0,0,17.0,98,1021.1,284.0,2.7
2,2018-12-31 04:00:00,2018-12-31 03:00:00,7.6,0.0,0,0,16.6,98,1020.9,288.0,2.6
3,2018-12-31 05:00:00,2018-12-31 04:00:00,7.6,0.0,0,0,16.5,98,1020.7,296.0,2.5
4,2018-12-31 06:00:00,2018-12-31 05:00:00,7.5,0.0,0,0,16.4,98,1020.5,303.0,2.5


## II - Feature engineering

Some new features can be created from the data variables contained in our dataset

In [17]:
# Create time and date features from the PeriodEnd variable
df.insert(1, 'YearPE', df['PeriodEnd'].dt.year)
df.insert(2, 'MonthPE', df['PeriodEnd'].dt.month)
df.insert(3, 'HourPE', df['PeriodEnd'].dt.hour)

In [18]:
df.head(5)

,PeriodEnd,YearPE,MonthPE,HourPE,PeriodStart,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,WindSpeed
0,2018-12-31 02:00:00,2018,12,2,2018-12-31 01:00:00,7.8,0.0,0,0,17.6,98,1021.2,282.0,2.6
1,2018-12-31 03:00:00,2018,12,3,2018-12-31 02:00:00,7.7,0.0,0,0,17.0,98,1021.1,284.0,2.7
2,2018-12-31 04:00:00,2018,12,4,2018-12-31 03:00:00,7.6,0.0,0,0,16.6,98,1020.9,288.0,2.6
3,2018-12-31 05:00:00,2018,12,5,2018-12-31 04:00:00,7.6,0.0,0,0,16.5,98,1020.7,296.0,2.5
4,2018-12-31 06:00:00,2018,12,6,2018-12-31 05:00:00,7.5,0.0,0,0,16.4,98,1020.5,303.0,2.5


In [19]:
# Create time and date features from the PeriodStart variable
df.insert(5, 'YearPS', df['PeriodStart'].dt.year)
df.insert(6, 'MonthPS', df['PeriodStart'].dt.month)
df.insert(7, 'HourPS', df['PeriodStart'].dt.hour)

In [20]:
df.head(5)

,PeriodEnd,YearPE,MonthPE,HourPE,PeriodStart,YearPS,MonthPS,HourPS,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,WindSpeed
0,2018-12-31 02:00:00,2018,12,2,2018-12-31 01:00:00,2018,12,1,7.8,0.0,0,0,17.6,98,1021.2,282.0,2.6
1,2018-12-31 03:00:00,2018,12,3,2018-12-31 02:00:00,2018,12,2,7.7,0.0,0,0,17.0,98,1021.1,284.0,2.7
2,2018-12-31 04:00:00,2018,12,4,2018-12-31 03:00:00,2018,12,3,7.6,0.0,0,0,16.6,98,1020.9,288.0,2.6
3,2018-12-31 05:00:00,2018,12,5,2018-12-31 04:00:00,2018,12,4,7.6,0.0,0,0,16.5,98,1020.7,296.0,2.5
4,2018-12-31 06:00:00,2018,12,6,2018-12-31 05:00:00,2018,12,5,7.5,0.0,0,0,16.4,98,1020.5,303.0,2.5


In [21]:
# create a new feature for sunrise using the function below; fill time for sunrise
def time_to_sunrise(row):
    """
    A function for identifying time of sunrise.
    """
    value= 0
    if row['HourPS']== 5 and row['HourPE']== 6 and row['Radiation']> 0:
        value= '06:00:00'
    elif row['HourPS']== 6 and row['HourPE']== 7 and row['Radiation']> 0:
        value= '07:00:00'
    elif row['HourPS']== 7 and row['HourPE']== 8 and row['Radiation']> 0:
        value= '08:00:00'
    elif row['HourPS']== 8 and row['HourPE']== 9 and row['Radiation']> 0:
        value= '08:30:00' # Set 8.30 the highest avg value is 8.36 in January and Decemer
    return value

In [22]:
#Create a new column using the function above
df['Sunrise']= df.apply(time_to_sunrise, axis=1)
df.head(5)

,PeriodEnd,YearPE,MonthPE,HourPE,PeriodStart,YearPS,MonthPS,HourPS,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,WindSpeed,Sunrise
0,2018-12-31 02:00:00,2018,12,2,2018-12-31 01:00:00,2018,12,1,7.8,0.0,0,0,17.6,98,1021.2,282.0,2.6,0
1,2018-12-31 03:00:00,2018,12,3,2018-12-31 02:00:00,2018,12,2,7.7,0.0,0,0,17.0,98,1021.1,284.0,2.7,0
2,2018-12-31 04:00:00,2018,12,4,2018-12-31 03:00:00,2018,12,3,7.6,0.0,0,0,16.6,98,1020.9,288.0,2.6,0
3,2018-12-31 05:00:00,2018,12,5,2018-12-31 04:00:00,2018,12,4,7.6,0.0,0,0,16.5,98,1020.7,296.0,2.5,0
4,2018-12-31 06:00:00,2018,12,6,2018-12-31 05:00:00,2018,12,5,7.5,0.0,0,0,16.4,98,1020.5,303.0,2.5,0


In [23]:
df['Sunrise'].value_counts()

0           22595
08:30:00     1087
08:00:00     1054
07:00:00      775
06:00:00      574
Name: Sunrise, dtype: int64

In [24]:
# Create a new feature for sunset using the function below; fill time for sunset
# Sunset hours established based on average sunset time for each month in Paris
def time_to_sunset(row):
    """
    A function for identifying time of sunset.
    """
    value= 0
    if row['HourPS'] == 17 and row['HourPE'] == 18 and row['Radiation']> 0:
        value= '18:00:00'
    elif row['HourPS'] == 18 and row['HourPE']== 19 and row['Radiation']> 0:
        value= '19:00:00'
    elif row['HourPS'] == 19 and row['HourPE']== 20 and row['Radiation']> 0:
        value= '20:00:00'
    elif row['HourPS'] == 20 and row['HourPE']== 21 and row['Radiation']> 0:
        value= '21:00:00'
    elif row['HourPS'] == 21 and row['HourPE']== 22 and row['Radiation']> 0:
        value= '22:00:00'
    return value

In [25]:
#Create a new column using the function above
df['Sunset']= df.apply(time_to_sunset, axis=1)

In [26]:
df['Sunset'].value_counts()

0           24598
18:00:00      702
19:00:00      507
20:00:00      278
Name: Sunset, dtype: int64

In [27]:
# show summary of our dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26085 entries, 0 to 26084
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   PeriodEnd      26085 non-null  datetime64[ns]
 1   YearPE         26085 non-null  int64         
 2   MonthPE        26085 non-null  int64         
 3   HourPE         26085 non-null  int64         
 4   PeriodStart    26085 non-null  datetime64[ns]
 5   YearPS         26085 non-null  int64         
 6   MonthPS        26085 non-null  int64         
 7   HourPS         26085 non-null  int64         
 8   Temperature    26085 non-null  float64       
 9   Radiation      26085 non-null  float64       
 10  DNI            26085 non-null  int64         
 11  GHI            26085 non-null  int64         
 12  Precipitation  26085 non-null  float64       
 13  Humidity       26085 non-null  int64         
 14  Pressure       26085 non-null  float64       
 15  WindDirection  2608

In [28]:
# changing integer values to datetime format
df_new = df.loc[~((df['Radiation'] == 0) & (df['DNI'] == 0) & (df['GHI'] == 0)),:]

In [29]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13895 entries, 6 to 26078
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   PeriodEnd      13895 non-null  datetime64[ns]
 1   YearPE         13895 non-null  int64         
 2   MonthPE        13895 non-null  int64         
 3   HourPE         13895 non-null  int64         
 4   PeriodStart    13895 non-null  datetime64[ns]
 5   YearPS         13895 non-null  int64         
 6   MonthPS        13895 non-null  int64         
 7   HourPS         13895 non-null  int64         
 8   Temperature    13895 non-null  float64       
 9   Radiation      13895 non-null  float64       
 10  DNI            13895 non-null  int64         
 11  GHI            13895 non-null  int64         
 12  Precipitation  13895 non-null  float64       
 13  Humidity       13895 non-null  int64         
 14  Pressure       13895 non-null  float64       
 15  WindDirection  1389

In [30]:
# display 10 random samples of the dataframe
df_new.sample(10)

,PeriodEnd,YearPE,MonthPE,HourPE,PeriodStart,YearPS,MonthPS,HourPS,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,WindSpeed,Sunrise,Sunset
11583,2020-04-26 17:00:00,2020,4,17,2020-04-26 16:00:00,2020,4,16,20.2,90.0,650,345,13.6,29,998.8,344.0,2.6,0,0
11120,2020-04-07 10:00:00,2020,4,10,2020-04-07 09:00:00,2020,4,9,13.0,356.0,99,419,23.7,73,1017.8,61.0,2.9,0,0
4642,2019-07-12 12:00:00,2019,7,12,2019-07-12 11:00:00,2019,7,11,23.8,505.0,32,533,23.8,48,1004.3,296.0,4.2,0,0
13937,2020-08-02 19:00:00,2020,8,19,2020-08-02 18:00:00,2020,8,18,21.8,46.0,271,87,20.8,41,1005.9,305.0,4.2,0,19:00:00
21204,2021-06-01 14:00:00,2021,6,14,2021-06-01 13:00:00,2021,6,13,26.6,116.0,866,835,19.8,41,1000.5,102.0,3.9,0,0
9632,2020-02-05 10:00:00,2020,2,10,2020-02-05 09:00:00,2020,2,9,6.0,128.0,0,128,8.8,80,1025.6,4.0,1.5,0,0
14871,2020-09-10 17:00:00,2020,9,17,2020-09-10 16:00:00,2020,9,16,25.6,109.0,298,193,21.6,42,1006.6,328.0,1.4,0,0
1119,2019-02-15 17:00:00,2019,2,17,2019-02-15 16:00:00,2019,2,16,9.9,24.0,336,63,6.2,69,1012.5,151.0,3.1,0,0
5119,2019-08-01 09:00:00,2019,8,9,2019-08-01 08:00:00,2019,8,8,22.4,129.0,727,582,19.8,35,1007.6,161.0,0.6,08:30:00,0
1496,2019-03-03 10:00:00,2019,3,10,2019-03-03 09:00:00,2019,3,9,10.6,61.0,0,61,16.1,79,997.1,220.0,9.0,0,0


In [31]:
# convert datetime column of endperiod to just date
df_new['Date']= pd.to_datetime(df_new['PeriodEnd']).dt.normalize()

/var/folders/yr/dn66lq496bx16ymf6g3z61d40000gn/T/ipykernel_33998/3941231241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Date']= pd.to_datetime(df_new['PeriodEnd']).dt.normalize()


In [32]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13895 entries, 6 to 26078
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   PeriodEnd      13895 non-null  datetime64[ns]
 1   YearPE         13895 non-null  int64         
 2   MonthPE        13895 non-null  int64         
 3   HourPE         13895 non-null  int64         
 4   PeriodStart    13895 non-null  datetime64[ns]
 5   YearPS         13895 non-null  int64         
 6   MonthPS        13895 non-null  int64         
 7   HourPS         13895 non-null  int64         
 8   Temperature    13895 non-null  float64       
 9   Radiation      13895 non-null  float64       
 10  DNI            13895 non-null  int64         
 11  GHI            13895 non-null  int64         
 12  Precipitation  13895 non-null  float64       
 13  Humidity       13895 non-null  int64         
 14  Pressure       13895 non-null  float64       
 15  WindDirection  1389

In [33]:
# Convert the column (it's a string) to datetime type
dt_series= pd.to_datetime(df_new['Date'])

# Create datetime index passing the datetime series
dt_index= pd.DatetimeIndex(dt_series.values)

# Set the date as index
df_new1= df_new.set_index(dt_index)

In [34]:
df_new1.head(5)

,PeriodEnd,YearPE,MonthPE,HourPE,PeriodStart,YearPS,MonthPS,HourPS,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,WindSpeed,Sunrise,Sunset,Date
2018-12-31,2018-12-31 08:00:00,2018,12,8,2018-12-31 07:00:00,2018,12,7,7.6,1.0,0,1,16.2,98,1021.0,314.0,2.2,08:00:00,0,2018-12-31
2018-12-31,2018-12-31 09:00:00,2018,12,9,2018-12-31 08:00:00,2018,12,8,7.7,21.0,0,21,16.1,98,1021.3,319.0,2.1,08:30:00,0,2018-12-31
2018-12-31,2018-12-31 10:00:00,2018,12,10,2018-12-31 09:00:00,2018,12,9,7.9,32.0,0,32,15.5,97,1021.4,326.0,2.0,0,0,2018-12-31
2018-12-31,2018-12-31 11:00:00,2018,12,11,2018-12-31 10:00:00,2018,12,10,8.2,49.0,0,49,14.4,96,1021.2,333.0,2.1,0,0,2018-12-31
2018-12-31,2018-12-31 12:00:00,2018,12,12,2018-12-31 11:00:00,2018,12,11,8.4,45.0,0,45,13.3,94,1021.0,339.0,2.2,0,0,2018-12-31


In [35]:
df_new1.tail(5)

,PeriodEnd,YearPE,MonthPE,HourPE,PeriodStart,YearPS,MonthPS,HourPS,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,WindSpeed,Sunrise,Sunset,Date
2021-12-21,2021-12-21 12:00:00,2021,12,12,2021-12-21 11:00:00,2021,12,11,3.2,50.0,781,285,4.5,58,1012.5,79.0,2.7,0,0,2021-12-21
2021-12-21,2021-12-21 13:00:00,2021,12,13,2021-12-21 12:00:00,2021,12,12,3.7,50.0,775,277,4.4,54,1012.2,78.0,2.5,0,0,2021-12-21
2021-12-21,2021-12-21 14:00:00,2021,12,14,2021-12-21 13:00:00,2021,12,13,3.9,45.0,722,222,4.3,52,1011.7,73.0,2.6,0,0,2021-12-21
2021-12-21,2021-12-21 15:00:00,2021,12,15,2021-12-21 14:00:00,2021,12,14,3.9,36.0,589,131,4.1,51,1011.4,69.0,2.7,0,0,2021-12-21
2021-12-21,2021-12-21 16:00:00,2021,12,16,2021-12-21 15:00:00,2021,12,15,3.4,13.0,181,30,4.0,52,1011.4,69.0,2.7,0,0,2021-12-21


In [36]:
# Sample daily average for the solar and weather features
df_new1['Daily_radiation']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Radiation'].mean()
df_new1['Daily_DNI']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['DNI'].mean()
df_new1['Daily_GHI']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['GHI'].mean()
df_new1['Daily_Temp']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Temperature'].mean()
df_new1['Daily_Precip']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Precipitation'].mean()
df_new1['Daily_Humidity']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Humidity'].mean()
df_new1['Daily_Pressure']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['Pressure'].mean()
df_new1['Daily_WindDir']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['WindDirection'].mean()
df_new1['Daily_WindSpeed']= df_new1.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['WindSpeed'].mean()


In [37]:
# Drop irrelevant features
new_df= df_new1.drop(['PeriodStart', 'YearPS', 'HourPS', 'HourPE', 'Sunrise', 'Sunset', 'MonthPS', 'YearPE'], axis= 1)


In [38]:
# Display first 10 rows
new_df.head(10)

,PeriodEnd,MonthPE,Temperature,Radiation,DNI,GHI,Precipitation,Humidity,Pressure,WindDirection,...,Date,Daily_radiation,Daily_DNI,Daily_GHI,Daily_Temp,Daily_Precip,Daily_Humidity,Daily_Pressure,Daily_WindDir,Daily_WindSpeed
2018-12-31,2018-12-31 08:00:00,12,7.6,1.0,0,1,16.2,98,1021.0,314.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2018-12-31,2018-12-31 09:00:00,12,7.7,21.0,0,21,16.1,98,1021.3,319.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2018-12-31,2018-12-31 10:00:00,12,7.9,32.0,0,32,15.5,97,1021.4,326.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2018-12-31,2018-12-31 11:00:00,12,8.2,49.0,0,49,14.4,96,1021.2,333.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2018-12-31,2018-12-31 12:00:00,12,8.4,45.0,0,45,13.3,94,1021.0,339.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2018-12-31,2018-12-31 13:00:00,12,8.4,66.0,0,66,12.5,94,1020.8,341.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2018-12-31,2018-12-31 14:00:00,12,7.9,40.0,0,40,12.0,93,1020.7,338.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2018-12-31,2018-12-31 15:00:00,12,7.5,36.0,0,36,11.5,93,1020.6,333.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2018-12-31,2018-12-31 16:00:00,12,7.1,20.0,0,20,11.0,93,1020.6,325.0,...,2018-12-31,34.444444,0.0,34.444444,7.855556,13.611111,95.111111,1020.955556,329.777778,1.844444
2019-01-01,2019-01-01 08:00:00,1,6.2,1.0,0,1,9.2,96,1019.2,289.0,...,2019-01-01,35.888889,0.0,35.888889,6.900000,9.833333,89.888889,1019.022222,307.888889,3.522222


In [39]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13895 entries, 2018-12-31 to 2021-12-21
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PeriodEnd        13895 non-null  datetime64[ns]
 1   MonthPE          13895 non-null  int64         
 2   Temperature      13895 non-null  float64       
 3   Radiation        13895 non-null  float64       
 4   DNI              13895 non-null  int64         
 5   GHI              13895 non-null  int64         
 6   Precipitation    13895 non-null  float64       
 7   Humidity         13895 non-null  int64         
 8   Pressure         13895 non-null  float64       
 9   WindDirection    13895 non-null  float64       
 10  WindSpeed        13895 non-null  float64       
 11  Date             13895 non-null  datetime64[ns]
 12  Daily_radiation  13895 non-null  float64       
 13  Daily_DNI        13895 non-null  float64       
 14  Daily_GHI        1389

In [40]:
# Drop duplicate observations exceept the last value
new_df= new_df[~new_df.Date.duplicated(keep='last')]

In [41]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1087 entries, 2018-12-31 to 2021-12-21
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PeriodEnd        1087 non-null   datetime64[ns]
 1   MonthPE          1087 non-null   int64         
 2   Temperature      1087 non-null   float64       
 3   Radiation        1087 non-null   float64       
 4   DNI              1087 non-null   int64         
 5   GHI              1087 non-null   int64         
 6   Precipitation    1087 non-null   float64       
 7   Humidity         1087 non-null   int64         
 8   Pressure         1087 non-null   float64       
 9   WindDirection    1087 non-null   float64       
 10  WindSpeed        1087 non-null   float64       
 11  Date             1087 non-null   datetime64[ns]
 12  Daily_radiation  1087 non-null   float64       
 13  Daily_DNI        1087 non-null   float64       
 14  Daily_GHI        1087 

In [42]:
print(f"Our dataset contains {new_df.shape[0]} observations and {df.shape[1]} columns")

Our dataset contains 1087 observations and 19 columns


In [43]:
# List out the column names
new_df.columns

Index(['PeriodEnd', 'MonthPE', 'Temperature', 'Radiation', 'DNI', 'GHI',
       'Precipitation', 'Humidity', 'Pressure', 'WindDirection', 'WindSpeed',
       'Date', 'Daily_radiation', 'Daily_DNI', 'Daily_GHI', 'Daily_Temp',
       'Daily_Precip', 'Daily_Humidity', 'Daily_Pressure', 'Daily_WindDir',
       'Daily_WindSpeed'],
      dtype='object')

In [44]:
new_df['PeriodEnd'].values

array(['2018-12-31T16:00:00.000000000', '2019-01-01T16:00:00.000000000',
       '2019-01-02T16:00:00.000000000', ...,
       '2021-12-19T16:00:00.000000000', '2021-12-20T16:00:00.000000000',
       '2021-12-21T16:00:00.000000000'], dtype='datetime64[ns]')

In [45]:
# Create a list with the day of the year
list_day_year = []
for i in new_df['PeriodEnd'].values:
    day_year = pd.Period(i, freq='D')
    a = day_year.day_of_year
    list_day_year.append(a)

In [46]:
#Check the 1st days in our liste
list_day_year[0:20]

[365, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [47]:
new_df['Day_of_yr'] = list_day_year

In [48]:
# Check if dates are coherent looking at the maximum value
new_df['Day_of_yr'].max()

366

In [49]:
# Create a list with the season of each observations based on the day of the year
seasons = []
for i in new_df['Day_of_yr']:
    if i <=78:
        b = "Winter"
        seasons.append(b)
    elif i >= 79 and i <=171:
        b = "Spring"
        seasons.append(b)
    elif i >= 172 and i <=265:
        b = "Summer"
        seasons.append(b) 
    elif i >= 266 and i <=354:
        b = "Fall"
        seasons.append(b)
    if i >= 355:
        b = "Winter"
        seasons.append(b)

In [50]:
# Create the Season variable for our list
new_df['Season'] = seasons

In [51]:
new_df[['MonthPE', 'Date','Day_of_yr','Season']].tail(60)

,MonthPE,Date,Day_of_yr,Season
2021-10-23,10,2021-10-23,296,Fall
2021-10-24,10,2021-10-24,297,Fall
2021-10-25,10,2021-10-25,298,Fall
2021-10-26,10,2021-10-26,299,Fall
2021-10-27,10,2021-10-27,300,Fall
2021-10-28,10,2021-10-28,301,Fall
2021-10-29,10,2021-10-29,302,Fall
2021-10-30,10,2021-10-30,303,Fall
2021-10-31,10,2021-10-31,304,Fall
2021-11-01,11,2021-11-01,305,Fall


In [52]:
# select column features
final_df= new_df[['MonthPE', 'Date', 'Season','Daily_Temp','Daily_Precip', 'Daily_Humidity', 
                  'Daily_Pressure','Daily_WindDir','Daily_WindSpeed','Daily_DNI',
                  'Daily_GHI','Daily_radiation']]

A quick sanity check before saving our dataset, to make sure there are no null values

In [53]:
# Check missing values
final_df.isnull().sum()

MonthPE            0
Date               0
Season             0
Daily_Temp         0
Daily_Precip       0
Daily_Humidity     0
Daily_Pressure     0
Daily_WindDir      0
Daily_WindSpeed    0
Daily_DNI          0
Daily_GHI          0
Daily_radiation    0
dtype: int64

In [54]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1087 entries, 2018-12-31 to 2021-12-21
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   MonthPE          1087 non-null   int64         
 1   Date             1087 non-null   datetime64[ns]
 2   Season           1087 non-null   object        
 3   Daily_Temp       1087 non-null   float64       
 4   Daily_Precip     1087 non-null   float64       
 5   Daily_Humidity   1087 non-null   float64       
 6   Daily_Pressure   1087 non-null   float64       
 7   Daily_WindDir    1087 non-null   float64       
 8   Daily_WindSpeed  1087 non-null   float64       
 9   Daily_DNI        1087 non-null   float64       
 10  Daily_GHI        1087 non-null   float64       
 11  Daily_radiation  1087 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 110.4+ KB


In [55]:
# Save our cleaned dataset for later use
#final_df.to_csv("solarcast_df_clean281221.csv")